## 1. Configuration

* data_dir: directory of the dataset
* save_dir: directory of the model
* labeling_method: 'NPWE4i' or 'NPWEf' or 'DDOGCHOi'
* training_testing_scheme: 1 or 2 or 3  
* data_case:  
    * if training_testing_scheme = 1, it is the noise structure to be examined (choose any number from 1 to 12)  
    * if training_testing_scheme = 2 or 3, it is the viewing image plane to be examined (1 for transverse plane, 2 for longitudinal plane)


In [58]:
data_dir = 'D:/DATA/'
save_dir = 'D:/RESULTS/'
labeling_method = 'NPWE4i'
training_testing_scheme = 1
data_case = 1

model_name = labeling_method + '_scheme' + str(training_testing_scheme)+'_case'+str(data_case)

## 2. Loading Training Sets

* The total datasets are divided into training, validation, and testing sets with the ratio 2:1:1.

* Note 1: The dimension order of .mat file is originally (rows, cols, depth, batch)  
* Note 2: The dimension order of .mat file (version 7.3) is reversed when loaded with h5py, which is (batch, depth, cols, rows)  
* Note 3: The shape of 3D data for tensorflow is (batch, rows, cols, depth, channels).  



In [73]:
import numpy as np
import h5py
from scipy import io

train_list = {
    1: [data_case],
    2: list(range(6*(data_case-1)+1,6*data_case+1)),
    3: list(range(6*(data_case-1)+1,6*data_case+1))
}[training_testing_scheme]

N_PA = 4000 # number of image pairs (g1 and g0)
N_TR = 2000 # number of training pairs
N_VA = 1000 # number of validation pairs
N_TE = 1000 # number of testing pairs

input_processing = True # make DC component to zero

random_seed_number = 3 # shuffling training and validation sets.
np.random.seed(random_seed_number)
idx_seq1 = np.random.permutation(N_TR+N_VA)
idx_seq0 = np.random.permutation(N_TR+N_VA)

for i in range(0,len(train_list)):

    filename = data_dir+'N'+str(train_list[i])+'_g1.mat'
    f = h5py.File(filename,'r')
    g1_temp = np.asarray(f['g1'])
    
    filename = data_dir+'N'+str(train_list[i])+'_g0.mat'
    f = h5py.File(filename,'r')
    g0_temp = np.asarray(f['g0'])
    
    filename = data_dir+'label_N'+str(train_list[i])+'_'+labeling_method+'.mat'
    f = io.loadmat(filename)
    t1_temp = np.asarray(f['t1'])
    t1_temp = np.transpose(t1_temp, (1,0))
    t0_temp = np.asarray(f['t0'])
    t0_temp = np.transpose(t0_temp, (1,0))
        
    g1_temp = g1_temp[idx_seq1,:,:]
    t1_temp = t1_temp[idx_seq1]
    g0_temp = g0_temp[idx_seq0,:,:]
    t0_temp = t0_temp[idx_seq0]
            
    if i == 0:
        imgs = np.concatenate((g1_temp[:N_TR,:,:],g0_temp[:N_TR,:,:]),axis=0)
        imgs_val = np.concatenate((g1_temp[N_TR:N_TR+N_VA,:,:],g0_temp[N_TR:N_TR+N_VA,:,:]),axis=0)
        
        labels = np.concatenate((t1_temp[:N_TR],t0_temp[:N_TR]),axis=0)
        labels_val = np.concatenate((t1_temp[N_TR:N_TR+N_VA],t0_temp[N_TR:N_TR+N_VA]),axis=0)
        
    else:
        imgs = np.concatenate((imgs, g1_temp[:N_TR,:,:],g0_temp[:N_TR,:,:]), axis = 0)
        imgs_val = np.concatenate((imgs_val, g1_temp[N_TR:N_TR+N_VA,:,:],g0_temp[N_TR:N_TR+N_VA,:,:]), axis = 0)
        
        labels = np.concatenate((labels, t1_temp[:N_TR],t0_temp[:N_TR]), axis = 0) 
        labels_val = np.concatenate((labels_val, t1_temp[N_TR:N_TR+N_VA],t0_temp[N_TR:N_TR+N_VA]), axis = 0) 

imgs = np.transpose(imgs, (0,2,1))
imgs_val = np.transpose(imgs_val, (0,2,1))
        
imgs = imgs[...,np.newaxis]
imgs_val = imgs_val[...,np.newaxis]

if input_processing:
    print('input_processing == true.')
    for i in range(0, imgs.shape[0]):
        imgs[i,:,:,0] = imgs[i,:,:,0] - np.mean(imgs[i,:,:,0])
        if i < imgs_val.shape[0]:
            imgs_val[i,:,:,0] = imgs_val[i,:,:,0] - np.mean(imgs_val[i,:,:,0])
            
if labeling_method == 'NPWE4i' or labeling_method == 'NPWEf':
    labels = labels*1000
    labels_val = labels_val*1000
    
print(f"Training image shape: {imgs.shape}")
print(f"Training label shape: {labels.shape}")

print(f"Validation image shape: {imgs_val.shape}")
print(f"Validation label shape: {labels_val.shape}")

input_processing == true.
Training image shape: (4000, 129, 129, 1)
Training label shape: (4000, 1)
Validation image shape: (2000, 129, 129, 1)
Validation label shape: (2000, 1)


## 3.1. CNN Model Observer

* Keras implementation of the three-layer CNN.  


In [75]:
import keras
from keras.models import Model
from keras.layers import Activation, Input, LeakyReLU, ELU, GlobalAveragePooling2D, Conv1D, Conv2D, Dense, Softmax, Flatten, UpSampling2D
from keras.layers import Dropout, BatchNormalization, Reshape, SpatialDropout2D, GaussianNoise, concatenate, MaxPooling2D, ReLU, add, concatenate
from keras import regularizers
from keras import backend as K
from keras.initializers import TruncatedNormal, glorot_normal, glorot_uniform, he_normal, he_uniform, lecun_normal, lecun_uniform

def CNN_Observer(SIZE_FILTER = [5], NUM_CONV = [16], CONV_STRIDES = [1], NUM_FC = [], BN = False, ALPFA_CONV = 0.5, ALPFA_FC = 0, PAD = 'same', GAP = False):
    INPUT_SIZE = 129
        
    img_input = Input(shape=(INPUT_SIZE,INPUT_SIZE,1), name='input')
    for i in range(0,len(NUM_CONV)):    
        if i == 0:
            x = Conv2D(NUM_CONV[i], (SIZE_FILTER[i],SIZE_FILTER[i]), strides=CONV_STRIDES[i], 
                       padding=PAD, 
#                        kernel_initializer=TruncatedNormal(mean=0.0,stddev=0.05),
#                        kernel_regularizer=regularizers.l1(0.01),
                       name='conv'+str(i)
                      )(img_input)
        else:
            x = Conv2D(NUM_CONV[i], (SIZE_FILTER[i],SIZE_FILTER[i]), strides=CONV_STRIDES[i], 
                       padding=PAD, 
#                        kernel_initializer=TruncatedNormal(mean=0.0,stddev=0.05),
#                        kernel_regularizer=regularizers.l1(0.01),
                       name='conv'+str(i)
                      )(x)
        
        x = LeakyReLU(alpha=ALPFA_CONV, name='lrelu_conv'+str(i))(x)  
        if BN == True:
            x = BatchNormalization(name='bn_conv'+str(i))(x)
  
    if GAP == True:
        x = GlobalAveragePooling2D(name='gap')(x)
    else:
        x = Flatten(name='flattening')(x)
    
    for i in range(0,len(NUM_FC)):
        x = Dense(NUM_FC[i], 
#                   kernel_initializer=TruncatedNormal(mean=0.0,stddev=0.05),
#                   kernel_regularizer=regularizers.l2(0.01),
                  name='fc'+str(i))(x)
            
        x = LeakyReLU(alpha=ALPFA_FC, name='lrelu_fc'+str(i))(x)
        if BN == True:
            x = BatchNormalization(name='bn_fc'+str(i))(x)
        
    output_value = Dense(1, 
#                          kernel_initializer=TruncatedNormal(mean=0.0,stddev=0.05),
#                          kernel_regularizer=regularizers.l2(0.01),
                         name='output_value')(x)
    
    observer_model = Model(inputs=img_input, outputs=output_value, name='Observer')

    return observer_model

model_4AFC = CNN_Observer(SIZE_FILTER = [13,13], NUM_CONV = [2,4], CONV_STRIDES = [2,2], NUM_FC = [], BN = False, ALPFA_CONV = 0.0, ALPFA_FC = 0.0, PAD = 'same', GAP=False)
model_4AFC.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 129, 129, 1)       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 65, 65, 2)         340       
_________________________________________________________________
lrelu_conv0 (LeakyReLU)      (None, 65, 65, 2)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 33, 33, 4)         1356      
_________________________________________________________________
lrelu_conv1 (LeakyReLU)      (None, 33, 33, 4)         0         
_________________________________________________________________
flattening (Flatten)         (None, 4356)              0         
_________________________________________________________________
output_value (Dense)         (None, 1)                 4357      
Total para

## 3.2. CapsNet Model Observer

* Keras implementation of the three-layer CapsNet.  
* If you want to use CapsNet, please uncomment and run following codes 

In [65]:
# import keras
# import numpy as np
# from keras.models import Model
# from keras.layers import Conv2D, Dense, Input, Reshape, Lambda, Layer, Flatten
# from keras import backend as K
# import tensorflow as tf
# from keras import initializers
    
# # Reference: https://github.com/TheAILearner/Capsule-Network/blob/master/Capsule%20Network.ipynb
# class DigitCapsuleLayer(Layer):
#     # creating a layer class in keras
#     def __init__(self, Nout = 1, Nin = 33*33*2, Din = 2, Dout = 4, **kwargs):
#         super(DigitCapsuleLayer, self).__init__(**kwargs)
#         self.kernel_initializer = initializers.get('glorot_uniform')
#         self.Nout = Nout
#         self.Nin = Nin
#         self.Din = Din
#         self.Dout = Dout
    
#     def build(self, input_shape): 
#         # initialize weight matrix for each capsule in lower layer
#         self.W = self.add_weight(shape = [self.Nout, self.Nin, self.Dout, self.Din], initializer = self.kernel_initializer, name = 'weights')
#         self.built = True
    
#     def call(self, inputs):
#         inputs = K.expand_dims(inputs, 1)
#         inputs = K.tile(inputs, [1, self.Nout, 1, 1])
#         # matrix multiplication b/w previous layer output and weight matrix
#         inputs = K.map_fn(lambda x: K.batch_dot(x, self.W, [2, 3]), elems=inputs)
#         b = tf.zeros(shape = [K.shape(inputs)[0], self.Nout, self.Nin])
#         # routing algorithm with updating coupling coefficient c, using scalar product b/w input capsule and output capsule
#         for i in range(3-1):
#             c = tf.nn.softmax(b, axis=1)
#             s = K.batch_dot(c, inputs, [2, 2])
# #             v = squashing(s)
#             v = s
#             b = b + K.batch_dot(v, inputs, [2,3])
            
#         return v 
#     def compute_output_shape(self, input_shape):
#         return tuple([None, self.Nout, self.Dout])

# def squashing(inputs):
#     squared_norm = K.sum(K.square(inputs), axis = -1, keepdims = True)
#     return squared_norm/(1+squared_norm) * inputs / K.sqrt(squared_norm+K.epsilon()) 

# def norm(inputs):
#     return K.sqrt(K.sum(K.square(inputs), axis = -1, keepdims = False) + K.epsilon())

# def CapsNet_Observer():
    
#     img_input = Input(shape=(129,129,1), name='input')
    
#     # convolution layer
#     x = Conv2D(2, (13,13), strides = 2, activation = 'relu', padding = 'same', name='conv1')(img_input)
    
#     # 2-D capsule layer
#     x = Conv2D(4, (13,13), strides = 2, activation = 'relu', padding = 'same', name='conv2')(x)
#     x_reshaped = Reshape((33*33*2,2), name='reshape')(x)
    
#     # 4-D capsule layer
#     x_DigitCaps = DigitCapsuleLayer(name='DigitCaps')(x_reshaped)
    
#     output_value = Lambda(norm, name='norm')(x_DigitCaps)

#     observer_model =Model(inputs=img_input, outputs=output_value, name='Observer')
    
#     return observer_model

# model_4AFC = CapsNet_Observer()
# model_4AFC.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 129, 129, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 65, 65, 2)         340       
_________________________________________________________________
conv2 (Conv2D)               (None, 33, 33, 4)         1356      
_________________________________________________________________
reshape (Reshape)            (None, 2178, 2)           0         
_________________________________________________________________
DigitCaps (DigitCapsuleLayer (None, 1, 4)              17424     
_________________________________________________________________
norm (Lambda)                (None, 1)                 0         
Total params: 19,120
Trainable params: 19,120
Non-trainable params: 0
_________________________________________________________________


## 4. Training Model Observer

* save_model_address: file path name of the best model.
* save_history_address: file path name of learning curve.
* save_last_epoch_address: file path name of model at the last epoch

In [76]:
from keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
from datetime import datetime
from keras.utils import generic_utils
import math

save_model_address = save_dir+'model_'+model_name
save_history_address = save_dir+'history_'+model_name+'.npy'
save_last_epoch_address = save_dir+'last_model_'+model_name+'.h5'

LEARNING_RATE = 1e-3
BATCH_SIZE = 64
NUM_EPOCHS = 100
NUM_4AFC = 100
ITER = 100
human_Pc = np.asarray([0.8086, 0.7814, 0.7600, 0.8514, 0.7971, 0.7400, 0.8914, 0.8571, 0.8114, 0.9286, 0.7929, 0.7186])

opt = keras.optimizers.Adam(lr=LEARNING_RATE)
model_4AFC.compile(optimizer=opt,
                   loss = 'mean_squared_error'
                            )

print(f'The checkpoint is saved in the following address:\n{save_model_address}')

best_ValLoss = np.Inf
best_ValPc = np.inf
best_TrainLoss = np.inf
history = {}

num_samples = labels.shape[0]

for epochs in range(NUM_EPOCHS):
    print (f"{epochs} epoch ")
    progbar = generic_utils.Progbar(num_samples)
    
#     new_learing_rate = step_decay(epochs)
#     K.set_value(model_4AFC.optimizer.lr, new_learing_rate)  # set new lr
    
    np.random.seed(epochs)
    idx_seq = np.random.permutation(num_samples)
    imgs_ = imgs[idx_seq,...]
    labels_ = labels[idx_seq,...]
    
    train_loss_list = []
    for iters in range(math.ceil(num_samples/(BATCH_SIZE))): # iteration within epoch

        last_idx = min((iters+1)*BATCH_SIZE, num_samples)
        train_loss_temp = model_4AFC.train_on_batch(
                                                    imgs_[iters*BATCH_SIZE:last_idx,...], 
                                                    labels_[iters*BATCH_SIZE:last_idx,...]           
                                                   )
        train_loss_temp = [train_loss_temp]
        train_loss_list.append(train_loss_temp)
        
        if last_idx == num_samples:
            progbar.add(last_idx-iters*BATCH_SIZE-2, values=[(n, v) for (n,v) in zip(model_4AFC.metrics_names, train_loss_temp)])
        else:
            progbar.add(last_idx-iters*BATCH_SIZE, values=[(n, v) for (n,v) in zip(model_4AFC.metrics_names, train_loss_temp)])
    
    train_loss_mean = np.asarray(train_loss_list)
    train_loss_mean = np.mean(train_loss_mean,axis=0)
    
    for i in range(len(train_loss_mean)):
        k = model_4AFC.metrics_names[i]
        history.setdefault(k, []).append(train_loss_mean[i])


    # validation RMSE in Pc
    val_pred = model_4AFC.predict(imgs_val)

    Pc_CNN = np.zeros((len(train_list), ITER))
    Pc_target = np.zeros((len(train_list),))
    for k in range(len(train_list)):
        Pc_target[k] = human_Pc[int(train_list[k])-1]
        np.random.seed(1)
        t1_raw = val_pred[2*k*N_VA:(2*k+1)*N_VA]
        t0_raw = val_pred[(2*k+1)*N_VA:2*(k+1)*N_VA] 

        for l in range(ITER):
            idx_seq1 = np.random.permutation(N_VA)
            idx_seq0 = np.random.permutation(N_VA)
            t1_raw = t1_raw[idx_seq1]
            t0_raw = t0_raw[idx_seq0]

            prediction_4AFC = np.concatenate((t1_raw[0:NUM_4AFC], 
                                              t0_raw[0:NUM_4AFC], 
                                              t0_raw[NUM_4AFC:NUM_4AFC*2], 
                                              t0_raw[NUM_4AFC*2:NUM_4AFC*3]),axis=1)

            answer_CNN = np.argmax(prediction_4AFC,axis=1)
            Pc_CNN[k, l] = np.sum(answer_CNN==0) / NUM_4AFC
            

    val_loss = np.sqrt(np.mean((np.mean(Pc_CNN, axis = 1) - Pc_target) **2))
    progbar.add(1, values=[('val_RMSE_in_Pc', val_loss)])
    
    history.setdefault('val_RMSE_in_Pc',[]).append(val_loss)

    # validation loss

    val_loss = model_4AFC.evaluate(imgs_val, 
                                   labels_val,
                                   batch_size=BATCH_SIZE,
                                   verbose=0)
    val_loss = [val_loss]
    progbar.add(1, values=[('val_'+n, v) for (n,v) in zip(model_4AFC.metrics_names, val_loss)])
    
    for i in range(len(val_loss)):
        k = 'val_'+model_4AFC.metrics_names[i]
        history.setdefault(k, []).append(val_loss[i])

    if epochs > 5:
        current_ValPc = history.get('val_RMSE_in_Pc')[-1]
        if current_ValPc < best_ValPc:
            best_ValPc = current_ValPc
            model_4AFC.save(save_model_address+'.h5', overwrite=True)
            print('Model Saved (ValPC) ')
        
#     current_ValLoss = history.get('val_loss')[-1]
#     if current_ValLoss < best_ValLoss:
#         best_ValLoss = current_ValLoss
#         model_4AFC.save(save_model_address+'_valLoss.h5', overwrite=True)
#         print('Model Saved (ValLoss)')
    
#     if epochs > 140:
#         current_TrainLoss = history.get('loss')[-1]
#         if current_TrainLoss < best_TrainLoss:
#             best_TrainLoss = current_TrainLoss
#             model_4AFC.save(save_model_address+'_trainLoss.h5', overwrite=True)
#             print('Model Saved (TrainLoss)')

print(f'\nThe last model is saved in the following file path:\n{save_last_epoch_address}')
model_4AFC.save(save_last_epoch_address)
np.save(save_history_address, history)

# print(history.keys())

The checkpoint is saved in the following address:
D:/RESULTS/model_NPWE4i_scheme1_case1
0 epoch 
4000/4000 [==============================] - 2s 571us/step - loss: 5.7780 - val_RMSE_in_Pc: 0.1078 - val_loss: 5.2566
1 epoch 
4000/4000 [==============================] - 2s 447us/step - loss: 3.7664 - val_RMSE_in_Pc: 0.0921 - val_loss: 2.1837
2 epoch 
4000/4000 [==============================] - 2s 450us/step - loss: 1.9426 - val_RMSE_in_Pc: 0.1092 - val_loss: 2.1118
3 epoch 
4000/4000 [==============================] - 2s 451us/step - loss: 1.7239 - val_RMSE_in_Pc: 0.1141 - val_loss: 1.7368
4 epoch 
4000/4000 [==============================] - 2s 452us/step - loss: 1.6004 - val_RMSE_in_Pc: 0.1150 - val_loss: 1.7174
5 epoch 
4000/4000 [==============================] - 2s 450us/step - loss: 1.5507 - val_RMSE_in_Pc: 0.1082 - val_loss: 1.6649
6 epoch 
4000/4000 [==============================] - 2s 457us/step - loss: 1.4908 - val_RMSE_in_Pc: 0.1105 - val_loss: 1.6422
Model Saved (ValPC) 
7 

4000/4000 [==============================] - 2s 450us/step - loss: 0.6165 - val_RMSE_in_Pc: 0.0529 - val_loss: 2.3220
61 epoch 
4000/4000 [==============================] - 2s 455us/step - loss: 0.5868 - val_RMSE_in_Pc: 0.0382 - val_loss: 2.3043
Model Saved (ValPC) 
62 epoch 
4000/4000 [==============================] - 2s 476us/step - loss: 0.5681 - val_RMSE_in_Pc: 0.0457 - val_loss: 2.3832
63 epoch 
4000/4000 [==============================] - 2s 455us/step - loss: 0.5593 - val_RMSE_in_Pc: 0.0338 - val_loss: 2.3646
Model Saved (ValPC) 
64 epoch 
4000/4000 [==============================] - 2s 465us/step - loss: 0.5379 - val_RMSE_in_Pc: 0.0390 - val_loss: 2.3875
65 epoch 
4000/4000 [==============================] - 2s 458us/step - loss: 0.5393 - val_RMSE_in_Pc: 0.0368 - val_loss: 2.4279
66 epoch 
4000/4000 [==============================] - 2s 456us/step - loss: 0.5143 - val_RMSE_in_Pc: 0.0230 - val_loss: 2.4148
Model Saved (ValPC) 
67 epoch 
4000/4000 [==============================

## 5. Loading Testing Sets


In [78]:
test_list = {
    1: [data_case],
    2: list(range(6*(data_case-1)+1,6*data_case+1)),
    3: list(range(12-6*data_case+1,12-6*(data_case-1)+1))
}[training_testing_scheme]

N_PA = 4000 # number of image pairs (g1 and g0)
N_TR = 2000 # number of training pairs
N_VA = 1000 # number of validation pairs
N_TE = 1000 # number of testing pairs

input_processing = True # make DC component to zero

random_seed_number = 3 # shuffling training and validation sets.
np.random.seed(random_seed_number)
idx_seq1 = np.random.permutation(N_TR+N_VA)
idx_seq0 = np.random.permutation(N_TR+N_VA)

for i in range(0,len(test_list)):

    filename = data_dir+'N'+str(test_list[i])+'_g1.mat'
    f = h5py.File(filename,'r')
    g1_temp = np.asarray(f['g1'])
    
    filename = data_dir+'N'+str(test_list[i])+'_g0.mat'
    f = h5py.File(filename,'r')
    g0_temp = np.asarray(f['g0'])
    
    filename = data_dir+'label_N'+str(test_list[i])+'_'+labeling_method+'.mat'
    f = io.loadmat(filename)
    t1_temp = np.asarray(f['t1'])
    t1_temp = np.transpose(t1_temp, (1,0))
    t0_temp = np.asarray(f['t0'])
    t0_temp = np.transpose(t0_temp, (1,0))
            
    if i == 0:
        imgs_test = np.concatenate((g1_temp[N_TR+N_VA:N_TR+N_VA+N_TE,:,:],g0_temp[N_TR+N_VA:N_TR+N_VA+N_TE,:,:]),axis=0)
        labels_test = np.concatenate((t1_temp[N_TR+N_VA:N_TR+N_VA+N_TE],t0_temp[N_TR+N_VA:N_TR+N_VA+N_TE]),axis=0)
        
    else:
        imgs_test = np.concatenate((imgs_test, g1_temp[N_TR+N_VA:N_TR+N_VA+N_TE,:,:],g0_temp[N_TR+N_VA:N_TR+N_VA+N_TE,:,:]), axis = 0)
        labels_test = np.concatenate((labels_test, t1_temp[N_TR+N_VA:N_TR+N_VA+N_TE],t0_temp[N_TR+N_VA:N_TR+N_VA+N_TE]), axis = 0) 


imgs_test = np.transpose(imgs_test, (0,2,1)) 
imgs_test = imgs_test[...,np.newaxis]

if input_processing:
    print('input_processing == true.')
    for i in range(0, imgs_test.shape[0]):
        imgs_test[i,:,:,0] = imgs_test[i,:,:,0] - np.mean(imgs_test[i,:,:,0])
        
print(f"Testing image shape: {imgs_test.shape}")
print(f"Testing label shape: {labels_test.shape}")

input_processing == true.
Testing image shape: (2000, 129, 129, 1)
Testing label shape: (2000, 1)


## 6. Loading Model Observer

* load_model_address: file path name of your model observer.

In [80]:
from keras.models import load_model, Model

load_model_address = save_model_address

model_loaded =  load_model(load_model_address+'.h5',
                           custom_objects={'DigitCapsuleLayer': DigitCapsuleLayer, 'norm': norm, 'squashing': squashing})

model_loaded.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 129, 129, 1)       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 65, 65, 2)         340       
_________________________________________________________________
lrelu_conv0 (LeakyReLU)      (None, 65, 65, 2)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 33, 33, 4)         1356      
_________________________________________________________________
lrelu_conv1 (LeakyReLU)      (None, 33, 33, 4)         0         
_________________________________________________________________
flattening (Flatten)         (None, 4356)              0         
_________________________________________________________________
output_value (Dense)         (None, 1)                 4357      
Total para

## 7. Testing Model Observer

* The model observer performs the 4-AFC detection tasks where it choose the image with the highest decision variable as a signal-present image among one signal-present and three signal-absent images in each trial.

---

* save_results_address: file path name of 4-AFC testing results.

In [89]:
import random

save_results_address = load_model_address+'_4AFC'

prediction_test = model_loaded.predict(imgs_test)

human_Pc = np.asarray([0.8086, 0.7814, 0.7600, 0.8514, 0.7971, 0.7400, 0.8914, 0.8571, 0.8114, 0.9286, 0.7929, 0.7186])

NUM_4AFC = 100
ITER = 100

Pc_CNN = np.zeros((len(test_list), ITER))
Pc_NPWE = np.zeros((len(test_list), ITER))

o_CNN = []
o_NPWE = []

random_seeds = 2

for j in range(0,len(test_list)):
    np.random.seed(random_seeds)
    for i in range(0,ITER):

        idx_seq1 = np.random.permutation(N_TE)
        idx_seq0 = np.random.permutation(N_TE)

        prediction_g1 = prediction_test[2*j*N_TE+idx_seq1] # Signal Present
        labels_g1 = labels_test[2*j*N_TE+idx_seq1]

        prediction_g0 = prediction_test[(2*j+1)*N_TE+idx_seq0] # Signal Absent
        labels_g0 = labels_test[(2*j+1)*N_TE+idx_seq0]

        prediction_4AFC = np.concatenate((prediction_g1[0:NUM_4AFC], 
                                          prediction_g0[0:NUM_4AFC], 
                                          prediction_g0[NUM_4AFC:NUM_4AFC*2], 
                                          prediction_g0[NUM_4AFC*2:NUM_4AFC*3]),axis=1)
        label_4AFC = np.concatenate((labels_g1[0:NUM_4AFC], 
                                     labels_g0[0:NUM_4AFC], 
                                     labels_g0[NUM_4AFC:NUM_4AFC*2], 
                                     labels_g0[NUM_4AFC*2:NUM_4AFC*3]),axis=1)

        answer_CNN = np.argmax(prediction_4AFC,axis=1)
        answer_NPWE = np.argmax(label_4AFC,axis=1)

        o_CNN = np.append(o_CNN, answer_CNN==0)
        o_NPWE = np.append(o_NPWE, answer_NPWE==0)

        Pc_CNN[j, i] = np.sum(answer_CNN==0) / NUM_4AFC
        Pc_NPWE[j, i] = np.sum(answer_NPWE==0) / NUM_4AFC

print(f'Pc of CNN = {np.mean(Pc_CNN, axis = 1)}')
print(f'Pc of human observers (target) = {human_Pc[[x - 1 for x in test_list]]}')


from scipy import io
io.savemat(save_results_address+'.mat',
           mdict={'output_test': prediction_test, 'label_test':labels_test, 'o_output': o_CNN, 'o_label': o_NPWE,'Pc_output': Pc_CNN, 'Pc_label':Pc_NPWE})

Pc of CNN = [0.8217]
Pc of human observers (target) = [0.8086]
